In [1]:
import copy
import datetime as dt
import json
import operator
import os

import iso8601
import pandas as pd

import bb_binary.parsing as bbb_p
import bb_videos_archiv.core as bb_va

In [2]:
def toggle(param):
    if param == 'left':
        return 'right'
    elif param == 'right':
        return 'left'
    elif param ==  0:
        return 1
    elif param == 1:
        return 0
    elif param == 2:
        return 3
    elif param == 3:
        return 2
    else:
        raise ValueError("Just the following values are accepted 'left', 'right', 0, 1, 2 and 3.")

In [3]:
def read_n_parse_json(path):
    df = pd.read_json(path)
    df['cam_id'] = df.apply(lambda row: bbb_p.parse_video_fname(row['start_video_name'])[0], axis=1)
    df['ts_start'] = df.apply(lambda row: bbb_p.parse_video_fname(row['start_video_name'])[1], axis=1)
    df['ts_end'] = df.apply(lambda row: bbb_p.parse_video_fname(row['end_video_name'])[2], axis=1)
    return df

In [4]:
def get_sorted_intervals(path_left, path_right):
    """Return dataframe, wich contains the intervals from `path_left` and `path_right` sorted by their timesamp
    of the starting frame.
    
    Args:
        path_left (str): path of the json file, which holds the intervals from step 5 of cam 5.
        path_right (str): path of the json file, which holds the intervals from step 5 of cam 5.
        cam_id_left (str): id of the left camera.
        cam_id_right (str): id of the right camera.     
    """
    data_cam_l = read_n_parse_json(path_left)
    data_cam_r = read_n_parse_json(path_right)
    intervals = pd.concat([data_cam_l, data_cam_r], axis=0, ignore_index=True)
    return intervals.sort_values(by='ts_start', ascending=False)   

In [5]:
def get_intersection_interval(sorted_data, cam_id_left, cam_id_right, thold):
    # Diese Funktion ist dafür um festzustellen welche Zeitintervalle als potentiell der gleiche Anfang angesehen
    # werden können.
    concat = False
    shared_intervals = []
    memory = None
    id_sh = 0
    for i, (idx_i, row_older) in enumerate(sorted_data.iterrows()):

        # Intervall besteht aus start und entpunkt, als gesamte Zeitangabe und dann jeweils die entsprechendenden Videos
        # für jede Kamera
        shared_interval = {}
        export_keys = ['stable', 'id', 'info', 'start_video_name', 'ts_start', 'cam_id']
        
        side = 'left' if row_older['cam_id'] == cam_id_left else 'right'

        shared_interval[side] = {}
        for key in export_keys:
            shared_interval[side][key] = row_older[key]
        shared_interval[toggle(side)] = memory

        if memory is not None:
            memory = None
            shared_interval['id'] = id_sh
            id_sh += 1
            shared_intervals.append(shared_interval)
            continue

        row_younger = sorted_data.iloc[i+1]

        if row_younger['cam_id'] == row_older['cam_id']: 
            shared_interval['id'] = id_sh
            id_sh += 1
            shared_intervals.append(shared_interval)
        else:
            if (row_older['ts_start'] - row_younger['ts_start']) < thold:
                memory = {}
                for key in export_keys:
                    memory[key] = row_older[key]
            else:
                shared_interval['id'] = id_sh
                id_sh += 1
                shared_intervals.append(shared_interval)
    return shared_intervals

In [6]:
def get_all_video_names_sorted(path):
    """Dies gibt ein dict zurück, welches als keys 'Cam_0' bis 'Cam_3' enthält.
    
    Unter jedem key ist eine geordnete liste aller filenames enthalten.
    """
    video_files = {}
    for __,__,files in os.walk(path):
        for file in files:
            if file.endswith('.mkv'):
                cam_id, __, __ = bbb_p.parse_video_fname(file)

                cam_id_str = 'Cam_{id}'.format(id=cam_id)
                if cam_id_str not in video_files:
                    video_files[cam_id_str] = []

                video_files[cam_id_str].append(file)

    for key in video_files.keys():
        video_files[key].sort()
    return video_files

In [7]:
def get_closest_video(ts, sorted_video_files, th):
    """Bestimmt das nächstliegende video, mittels th kann ein Schwellwert abgegeben werden,
    sodas falls der timtestamp 'ts' am ende eines Videos liegt. Das Nachfolgende Video ausgegeben werden.
    """
    time_delta = dt.timedelta(seconds=th)
    for i, video in enumerate(sorted_video_files):
        __, ts_start, ts_end = bbb_p.parse_video_fname(video)
        if ts_start < ts:
            if ts < ts_end:
                if ts_end - ts < time_delta and (i < len(sorted_video_files) -1):
                    return sorted_video_files[i+1]
                else:
                    return video
            else:
                continue
        else:
            return video

In [8]:
def extend_closest_videos(intervals, year, root_dir):
    """Sollte für ein Interval einer Kamera kein korrespondierendes Video der anderen Kamera existieren,
    wird hier nach dem nächstliegenden gesucht.
    """
    # va = bb_va.Video_Archiv(year, root_dir)
    video_files = get_all_video_names_sorted(root_dir)
    # intervals = copy.deepcopy(intervals_org)
    for i, interval in enumerate(intervals):
        for key in ['left', 'right']:
            if interval[key] is None:
                exist_key = toggle(key)
                non_exist_cam = toggle(interval[exist_key]['cam_id'])
                ts = interval[exist_key]['ts_start']
                ts = ts.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
                ts = iso8601.parse_date(ts)
                cam_id_str = 'Cam_{id}'.format(id=non_exist_cam)
                video = get_closest_video(ts, video_files[cam_id_str], 5)
                
                __, ts_start_video, __= bbb_p.parse_fname(video)
                time_delta = dt.timedelta(minutes=30)
                
                if (abs(ts_start_video-ts) < time_delta):
                    interval[key] = {}
                    interval[key]['start_video_name'] = video
                    interval[key]['stable'] = intervals[i-1][key]['stable']
                    interval[key]['cam_id'], interval[key]['ts_start']= bbb_p.parse_video_fname(video)[:2]
                    interval[key]['info'] = 'closest video'
    return intervals

In [9]:
def convert_2_valid_json(shared_intervals):
    shared_intervals_valid_json = []
    for interval in shared_intervals:
        valid_interval = copy.deepcopy(interval)
        valid_interval['id'] = len(shared_intervals) - valid_interval['id'] - 1
        for key in ['left', 'right']:
            if valid_interval[key] is not None:
                for key_ts in ['ts_start', 'ts_end']:
                    if key_ts in interval[key]:
                        valid_interval[key][key_ts] = interval[key][key_ts].strftime('%Y-%m-%dT%H:%M:%S.%fZ')

        for key_ts in ['ts_start', 'ts_end']:
            if key_ts in interval:
                valid_interval[key_ts] = interval[key_ts].strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        shared_intervals_valid_json.append(valid_interval)
    return shared_intervals_valid_json

In [10]:
def get_valid_intersection_intervals(path_left, path_right, cam_left, cam_right, thold, year, root_dir):
    sorted_data = get_sorted_intervals(path_left, path_right)
    shared_intervals = get_intersection_interval(sorted_data, cam_left, cam_right, thold)
    extended_shared_intervals = extend_closest_videos(shared_intervals, year, root_dir)
    return extended_shared_intervals[::-1]

In [11]:
def save_2_doc_n_locally(json_data, doc_path, filename):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, sort_keys=True, indent=2)
    
    with open(os.path.join(doc_path, filename), 'w') as fp:
        json.dump(json_data, fp, sort_keys=True, indent=2)

In [12]:
thold = dt.timedelta(seconds=90)
doc_path = '../docs'
root_dir = './videos_proxy'
year = '2016'

In [ ]:
path_left = '05c_Cam_0_intervals_ecc_stable_join_man.json'
path_right = '05c_Cam_1_intervals_ecc_stable_join_man.json'

cam_left = 0
cam_right = 1

In [ ]:
shared_intervals_valid_json_01 = get_valid_intersection_intervals(path_left, path_right, cam_left, cam_right, thold, year, root_dir)

In [ ]:
path_left = '05c_Cam_2_intervals_ecc_stable_join_man.json'
path_right = '05c_Cam_3_intervals_ecc_stable_join_man.json'
filename = '06_Cam_23_intervals_pair.json'
cam_left = 2
cam_right = 3

In [ ]:
shared_intervals_valid_json_23 = get_valid_intersection_intervals(path_left, path_right, cam_left, cam_right, thold, year, root_dir)

In [ ]:
video_files = get_all_video_names_sorted(root_dir)

In [ ]:
intervalle = []
for i in range(4):
    path = '05c_Cam_{cam_id}_intervals_ecc_stable_join_man.json'.format(cam_id=i)
    data = read_n_parse_json(path)
    intervalle.append(data)
df = pd.concat(intervalle, axis=0, ignore_index=True)
df = df.sort_values(by='ts_start', ascending=False)

In [ ]:
def get_stable_info(df, cam_id, ts_start, ts_end):
    result = df[((df.cam_id ==cam_id) & (df.ts_start <= ts_start) & (ts_end <= df.ts_end))]
    if len(result) == 1:
        return bool(result['stable'].values[0])
    else:
        print(ts_start)
        print(ts_end)
        print(cam_id)
        return True
        raise Exception('Es gibt mehr als ein Interval in dem dieses "beschnitte" Interval liegt, das sollte nicht möglich sein!')

In [ ]:
def extend_end_video_name(intervals, videos_files):
    """Bestimmt in Abhängigkeit zur Kamera das letzte Video eines Intervals."""
    for idx, interval in enumerate(intervals):
        for key in ['left', 'right']:
            
            if interval[key] is not None:
                cam_id_str = 'Cam_{id}'.format(id=interval[key]['cam_id'])
                curr_idx = idx + 1
                # falls das folgende Interval leer ist 
                while curr_idx  < len(intervals) and intervals[curr_idx][key] is None:
                    curr_idx = curr_idx + 1

                if curr_idx  < len(intervals) and intervals[curr_idx][key] is not None:
                    next_start_video = intervals[curr_idx][key]['start_video_name']
                    next_idx = video_files[cam_id_str].index(next_start_video)
                    end_idx = next_idx - 1
                    interval[key]['end_video_name'] = video_files[cam_id_str][end_idx]
                    __, __, interval[key]['ts_end'] = bbb_p.parse_video_fname(interval[key]['end_video_name'])
                    interval[key]['stable'] = get_stable_info(df, interval[key]['cam_id'],interval[key]['ts_start'], interval[key]['ts_end'])
                if curr_idx == len(intervals):
                    interval[key]['end_video_name'] = video_files[cam_id_str][-1]
                    __, __, interval[key]['ts_end'] = bbb_p.parse_video_fname(interval[key]['end_video_name'])
                    interval[key]['stable'] = get_stable_info(df, interval[key]['cam_id'],interval[key]['ts_start'], interval[key]['ts_end'])

In [ ]:
extend_end_video_name(shared_intervals_valid_json_01, video_files)

In [ ]:
extend_end_video_name(shared_intervals_valid_json_23, video_files)

In [ ]:
def extend_overall_interval(intervals):
    """Bestimmt Anfangs und Endzeitpunkt des gemeinsamen Intervals und markiert ein gemeinsames
    Interval als stable, wenn beide Videos stable sind."""
    def _set_interval_key(interval, key, relate):
        if interval['left'] is not None and interval['right'] is not None:
            if relate(interval['left'][key], interval['right'][key]):
                interval[key] = interval['left'][key]
            else:
                interval[key] = interval['right'][key]
        else:
            if interval['left'] is None:
                interval[key] = interval['right'][key]
            else:
                interval[key] = interval['left'][key]

    for i, interval in enumerate(intervals):
        _set_interval_key(interval, 'ts_start', operator.lt)
        _set_interval_key(interval, 'ts_end', operator.gt)
        if interval['left'] is None:
            interval['stable'] = interval['right']['stable']
        elif interval['right'] is None:
            interval['stable'] = interval['left']['stable']
        else:
            interval['stable'] = interval['left']['stable'] and interval['right']['stable']

In [ ]:
filename = '06_Cam_01_intervals_pair.json'
converted_01 = convert_2_valid_json(shared_intervals_valid_json_01)
extend_overall_interval(converted_01)
save_2_doc_n_locally(converted_01, doc_path, filename)

In [ ]:
filename = '06_Cam_23_intervals_pair.json'
converted_23 = convert_2_valid_json(shared_intervals_valid_json_23)
extend_overall_interval(converted_23)
save_2_doc_n_locally(converted_23, doc_path, filename)